# Serverless - Homework

In [1]:
from io import BytesIO
from urllib import request

from PIL import Image

import numpy as np

import tensorflow as tf
from tensorflow import keras
import tensorflow.lite as tflite

c:\Users\desol\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the converted model?
* 21 Mb
* 43 Mb
* 80 Mb
* 164 Mb

In [2]:
model = keras.models.load_model('bees-wasps.h5')

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('bees-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\desol\AppData\Local\Temp\tmpoonp8rmg\assets


INFO:tensorflow:Assets written to: C:\Users\desol\AppData\Local\Temp\tmpoonp8rmg\assets


The size of the converted model is 43Mb.

## Question 2

To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?
* 3
* 7
* 13
* 24

In [4]:
interpreter = tflite.Interpreter(model_path='bees-wasps.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
output_index

13

The output index for this model is 13.

In [5]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [6]:
img = download_image('https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg')

In [7]:
img = prepare_image(img, target_size=(150, 150))

C:\Users\desol\AppData\Local\Temp\ipykernel_76\2564224172.py:12: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


## Question 3

Now we need to turn the image into numpy array and pre-process it.

After the pre-processing, what's the value in the first pixel, the R channel?
* 0.3450980
* 0.5450980
* 0.7450980
* 0.9450980

In [8]:
def prepare_input(X):
    return X / 255.0

In [9]:
x = np.array(img, dtype='float32')
X = np.array([x])
X = prepare_input(X)

In [10]:
X[0, 0, 0, 0]

0.94509804

The value in the first pixel in the R channel is 0.9450980.

## Question 4

Now let's apply this model to this image. What's the output of the model?
* 0.258
* 0.458
* 0.658
* 0.858

In [11]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
pred = interpreter.get_tensor(output_index)
pred[0][0]

0.65898407

The output of the model is 0.658.